# Importing Libraries

In [30]:
!pip install --upgrade tensorflow --quiet
!pip install matplotlib --quiet
!pip install opencv-python --quiet
!pip install torch torchvision --quiet

In [31]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Loading Images

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),      
])

# Load dataset
data_dir = 'Corals' 
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the Model

In [33]:
import torch
import torch.nn as nn
from torchvision import models

# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)

# Modify the final layer for binary classification (2 classes)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/Users/kshitija/Library/CloudStorage/OneDrive-Personal/Major-1/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kshitija/Library/CloudStorage/OneDrive-Personal/Major-1/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/kshitija/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:08<00:00, 5.69MB/s]


In [34]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
print_every = 10  # Print loss every 10 batches

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()           
        outputs = model(images)        
        loss = criterion(outputs, labels) 
        loss.backward()                
        optimizer.step()              

        running_loss += loss.item()

        # Print loss every 'print_every' batches
        if (i + 1) % print_every == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {running_loss / print_every:.4f}')
            running_loss = 0.0


Epoch [1/10], Batch [10/281], Loss: 0.5969


KeyboardInterrupt: 

# Evaluate the model

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Saving the model Images

In [ ]:
model.save('frcnn_model.h5')